# Comparison of Replay Strategies in pyCLAD

This notebook compares the performance of `ReplayEnhancedStrategy` and `BalancedReservoirSamplingStrategy` from the `pyCLAD` library.
The comparison is performed across four datasets: Energy, NSL-KDD, UNSW, and Wind.
For each dataset, three different data scenarios are used: `random_anomalies`, `clustered_with_closest_assignment`, and `clustered_with_random_assignment`.

The notebook is divided into three main parts:
1.  **Setup**: Imports necessary libraries and defines the configuration for datasets, strategies, and output directories.
2.  **Run Experiments**: Executes the experiments for each combination of dataset, scenario, and strategy, saving the results to JSON files.
3.  **Analyze Results**: Loads the saved results, generates heatmaps for each experiment, and creates comparison heatmaps to visualize the performance difference between the two strategies.

In [1]:
import pathlib
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

# Datasets
from pyclad.data.datasets.unsw_dataset import UnswDataset
from pyclad.data.datasets.nsl_kdd_dataset import NslKddDataset
from pyclad.data.datasets.wind_energy_dataset import WindEnergyDataset
from pyclad.data.datasets.energy_plants_dataset import EnergyPlantsDataset

# Scenarios
from pyclad.scenarios.concept_aware import ConceptAwareScenario

from pyclad.metrics.continual.average_continual import ContinualAverage
from pyclad.metrics.continual.backward_transfer import BackwardTransfer
from pyclad.metrics.continual.forward_transfer import ForwardTransfer
# Models
from pyclad.models.adapters.pyod_adapters import LocalOutlierFactorAdapter

# Strategies
from pyclad.strategies.replay.replay import ReplayEnhancedStrategy
from pyclad.strategies.replay.candi import CandiStrategy

# Additional imports for replay strategies
from pyclad.strategies.replay.buffers.adaptive_balanced import AdaptiveBalancedReplayBuffer

from pyclad.strategies.replay.selection.random import RandomSelection

# Callback and metrics
from pyclad.callbacks.evaluation.concept_metric_evaluation import ConceptMetricCallback
from pyclad.callbacks.evaluation.memory_usage import MemoryUsageCallback
from pyclad.callbacks.evaluation.time_evaluation import TimeEvaluationCallback
from pyclad.metrics.base.roc_auc import RocAuc
from pyclad.output.json_writer import JsonOutputWriter
import time

# Configuration
DATASETS = {
    "energy": EnergyPlantsDataset,
    "nsl-kdd": NslKddDataset,
    "unsw": UnswDataset,
    "wind": WindEnergyDataset,
}

DATASET_TYPES = [
    "random_anomalies",
    "clustered_with_closest_assignment",
    "clustered_with_random_assignment",
]

max_buffer_size = 1000  # Maximum size for replay buffers  

STRATEGIES = {
    "replay_enhanced": lambda model: ReplayEnhancedStrategy(
        model,
        AdaptiveBalancedReplayBuffer(selection_method=RandomSelection(), max_size=max_buffer_size),
    ),
    "candi": lambda model: CandiStrategy(
        model, max_buffer_size=max_buffer_size, threshold_ratio=0.5, warmup_period=2, resize_new_regime=True
    ),
}


print("Setup complete.")

/Users/firepond/code/pyCLAD/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Setup complete.


### Run Experiments

The following cell runs the experiments. It iterates through each dataset and dataset type, and for each, it runs both the `ReplayEnhancedStrategy` and the `BalancedReservoirSamplingStrategy`. The results are saved in the `comparison_results` directory.

**Note:** This process can be time-consuming.

In [2]:
import time


def run_experiments():
    for dataset_name, dataset_class in DATASETS.items():
        for dataset_type in DATASET_TYPES:
            print(f"Running experiments for {dataset_name} - {dataset_type}")
            try:
                dataset = dataset_class(dataset_type=dataset_type)
            except Exception as e:
                print(f"Could not load dataset {dataset_name} with type {dataset_type}. Error: {e}")
                continue

            for strategy_name, strategy_builder in STRATEGIES.items():
                max_count = dataset.get_max_count()
                # peak memory usage 
                peak_size = max_count + max_buffer_size
                if strategy_name == "candi":
                    strategy_builder = lambda model: CandiStrategy(
                        model, max_buffer_size=peak_size//2, threshold_ratio=0.5, warmup_period=2, resize_new_regime=True
                    )
                start_time = time.time()
                model = LocalOutlierFactorAdapter()
                strategy = strategy_builder(model)

                callbacks = [
                    ConceptMetricCallback(
                        base_metric=RocAuc(),
                        metrics=[ContinualAverage(), BackwardTransfer(), ForwardTransfer()]
                    ),
                    TimeEvaluationCallback(),
                    MemoryUsageCallback(),
                ]
                scenario = ConceptAwareScenario(dataset, strategy=strategy, callbacks=callbacks)
                scenario.run()

                callbacks[0].print_continual_average()
                end_time = time.time()
                print(f"  Time taken: {end_time - start_time:.2f} seconds")

            print("-" * 20)
            
        print(f"Finished experiments for {dataset_name}.")
        print("=" * 40)

run_experiments()
print("All experiments finished.")

Running experiments for energy - random_anomalies
Fitting model on combined data of shape: (6000, 14)
Fitting model on combined data of shape: (7000, 14)
Fitting model on combined data of shape: (7000, 14)
Fitting model on combined data of shape: (6999, 14)
Fitting model on combined data of shape: (7000, 14)
Fitting model on combined data of shape: (7000, 14)
Fitting model on combined data of shape: (6996, 14)
Fitting model on combined data of shape: (6994, 14)
Fitting model on combined data of shape: (7000, 14)
Fitting model on combined data of shape: (6999, 14)
Lifelong Learning Metrics:
  ContinualAverage: 0.9556344212473573
  BackwardTransfer: -0.0060375968992248015
  ForwardTransfer: 0.5888242894056847
  Time taken: 18.42 seconds
Fitting model with 3500 samples
Fitting model with 7000 samples
Fitting model with 7001 samples
Fitting model with 7000 samples
Fitting model with 7000 samples
Fitting model with 7004 samples
Fitting model with 7000 samples
Fitting model with 7004 samples